In [1]:
import numpy as np
import pandas as pd

### Database Setup

In [2]:
%reload_ext sql

In [3]:
!psql postgresql://localhost/postgres -c 'DROP DATABASE IF EXISTS db'
!psql postgresql://localhost/postgres -c 'CREATE DATABASE db'

DROP DATABASE
CREATE DATABASE


In [4]:
df = pd.read_csv('covid.csv')
df

/var/folders/59/nt24bzqj5857m8c_dc4rtt440000gn/T/ipykernel_99483/2073813068.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('covid.csv')


,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,...,cdc_report_dtIs_year_end,cdc_report_dtIs_year_start,cdc_report_dtElapsed,pos_spec_dt_was_missing,onset_dt_was_missing,sex_was_missing,age_group_was_missing,Race and ethnicity (combined)_was_missing,pos_difference,onset_difference
0,2020-11-10,2020-11-10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,Unknown,No,...,False,False,1604966400,False,True,False,False,False,0.0,NaN
1,2020-11-14,2020-11-10,2020-11-10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,...,False,False,1605312000,False,False,False,False,False,4.0,4.0
2,2020-11-19,2020-11-10,2020-11-09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,...,False,False,1605744000,False,False,False,False,False,9.0,10.0
3,2020-11-14,2020-11-10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",Missing,Missing,No,...,False,False,1605312000,False,True,False,False,False,4.0,NaN
4,2020-11-13,2020-11-10,2020-11-10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,...,False,False,1605225600,False,False,False,False,False,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8405074,2020-10-22,NaN,NaN,Probable Case,Missing,30 - 39 Years,Unknown,No,Unknown,No,...,False,False,1603324800,True,True,False,False,False,NaN,NaN
8405075,2020-10-25,NaN,NaN,Laboratory-confirmed case,Missing,30 - 39 Years,Unknown,Missing,Missing,Missing,...,False,False,1603584000,True,True,False,False,False,NaN,NaN
8405076,2020-07-18,NaN,NaN,Laboratory-confirmed case,Missing,30 - 39 Years,Unknown,Missing,Missing,Missing,...,False,False,1595030400,True,True,False,False,False,NaN,NaN
8405077,2020-11-19,NaN,NaN,Laboratory-confirmed case,Missing,30 - 39 Years,Unknown,Missing,Missing,Missing,...,False,False,1605744000,True,True,False,False,False,NaN,NaN


In [5]:
df_sample = df.sample(n=1000000)

In [6]:
from sqlalchemy import create_engine

engine = create_engine("postgresql://localhost/db")
df_sample.to_sql(name="covid", con=engine)
engine.dispose()

In [7]:
%sql postgresql://localhost/db

Connecting to 'postgresql://localhost/db'

In [8]:
%sqlcmd tables

Name
covid


In [9]:
%%sql
SELECT *
FROM covid
LIMIT 10;

Running query in 'postgresql://localhost/db'

10 rows affected.

index,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn,cdc_report_dtYear,cdc_report_dtMonth,cdc_report_dtWeek,cdc_report_dtDay,cdc_report_dtDayofweek,cdc_report_dtDayofyear,cdc_report_dtIs_month_end,cdc_report_dtIs_month_start,cdc_report_dtIs_quarter_end,cdc_report_dtIs_quarter_start,cdc_report_dtIs_year_end,cdc_report_dtIs_year_start,cdc_report_dtElapsed,pos_spec_dt_was_missing,onset_dt_was_missing,sex_was_missing,age_group_was_missing,Race and ethnicity (combined)_was_missing,pos_difference,onset_difference
5805735,2020-08-24,2020-08-21,2020-08-21,Laboratory-confirmed case,Female,10 - 19 Years,"White, Non-Hispanic",No,Missing,Missing,Missing,2020,8,35,24,0,237,False,False,False,False,False,False,1598227200,False,False,False,False,False,3.0,3.0
1764197,2020-04-14,None,2020-04-02,Laboratory-confirmed case,Female,40 - 49 Years,"White, Non-Hispanic",Yes,Missing,No,Missing,2020,4,16,14,1,105,False,False,False,False,False,False,1586822400,True,False,False,False,False,None,12.0
5473015,2020-06-05,None,2020-06-05,Laboratory-confirmed case,Female,10 - 19 Years,Unknown,No,Missing,No,Missing,2020,6,23,5,4,157,False,False,False,False,False,False,1591315200,True,False,False,False,False,None,0.0
1520933,2020-08-05,None,2020-08-05,Laboratory-confirmed case,Male,40 - 49 Years,Unknown,No,Missing,No,Missing,2020,8,32,5,2,218,False,False,False,False,False,False,1596585600,True,False,False,False,False,None,0.0
1876645,2020-09-26,None,None,Laboratory-confirmed case,Female,40 - 49 Years,Missing,Unknown,Missing,Unknown,Missing,2020,9,39,26,5,270,False,False,False,False,False,False,1601078400,True,True,False,False,False,None,None
4479228,2020-04-25,None,2020-04-15,Laboratory-confirmed case,Female,80+ Years,"White, Non-Hispanic",No,Missing,Unknown,Unknown,2020,4,17,25,5,116,False,False,False,False,False,False,1587772800,True,False,False,False,False,None,10.0
6522734,2020-11-11,None,2020-11-10,Probable Case,Female,50 - 59 Years,Unknown,Missing,Missing,Missing,Missing,2020,11,46,11,2,316,False,False,False,False,False,False,1605052800,True,False,False,False,False,None,1.0
5681092,2020-11-17,None,2020-07-15,Laboratory-confirmed case,Female,10 - 19 Years,"Black, Non-Hispanic",Missing,Missing,Missing,Missing,2020,11,47,17,1,322,False,False,False,False,False,False,1605571200,True,False,False,False,False,None,125.0
5745210,2020-06-28,2020-06-30,2020-06-28,Laboratory-confirmed case,Female,10 - 19 Years,"White, Non-Hispanic",No,Unknown,No,No,2020,6,26,28,6,180,False,False,False,False,False,False,1593302400,False,False,False,False,False,-2.0,0.0
5184131,2020-10-28,None,None,Laboratory-confirmed case,Male,10 - 19 Years,Hispanic/Latino,Missing,Missing,Missing,Missing,2020,10,44,28,2,302,False,False,False,False,False,False,1603843200,True,True,False,False,False,None,None
